In [7]:
from dj_notebook import activate

plus = activate()

In [8]:
import sqlparse

from django.urls import reverse
from django.db import connection, models

from wagtail.models import Page
from wagtail.api.v2.filters import FieldsFilter
from rest_framework.filters import BaseFilterBackend
from rest_framework.test import APIRequestFactory
from taggit.managers import TaggableManager

from cast.api.views import FilteredPagesAPIViewSet

## How to Reproduce:

Click on:
- [python-podcast.de](https://python-podcast.de/api/wagtail/pages/?child_of=120&fields=html_detail%2Ccomments%2Ccomments_security_data%2Ccomments_are_enabled%2Cpodlove_players&slug=1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522&type=cast.Episode)
- [localhost](http://localhost:8000/api/wagtail/pages/?child_of=120&fields=html_detail%2Ccomments%2Ccomments_security_data%2Ccomments_are_enabled%2Cpodlove_players&slug=1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522&type=cast.Episode)
- [localhost simple](http://localhost:8000/api/wagtail/pages/?slug=1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522)

## Minimal Example

In [24]:
# bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
# request = rf.get(url)
# print(request.GET)
get_params = {"slug": "1\x00����%2527%2522"}
queryset = Page.objects.all().live()
queryset = queryset.filter(slug="1\x00����%2527%2522")
print(queryset.count())

#queryset = queryset.filter(**{"slug": request.GET["slug"]})
#print(get_formatted_sql(queryset))
#queryset.count()

DataError: PostgreSQL text fields cannot contain NUL (0x00) bytes

In [10]:
from django.test import RequestFactory

rf = RequestFactory()
bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
request = rf.get(f"?slug={bad_input}")
queryset = Page.objects.all().live()
queryset = queryset.filter(**{"slug": request.GET["slug"]})
print(request.GET["slug"])
queryset.count()

1 ����%2527%2522


DataError: PostgreSQL text fields cannot contain NUL (0x00) bytes

In [11]:
from rest_framework.test import APIRequestFactory

rf = APIRequestFactory()
bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
request = rf.get(f"?slug={bad_input}")
queryset = Page.objects.all().live()
queryset = queryset.filter(**{"slug": request.GET["slug"]})
print(request.GET["slug"])
queryset.count()

1 ����%2527%2522


DataError: PostgreSQL text fields cannot contain NUL (0x00) bytes

## Analyze Bug

In [12]:
def format_sql(query):
    return sqlparse.format(query, reindent=True, keyword_case='upper')

def get_formatted_sql(queryset):
    # Get the raw SQL query from the queryset
    sql, params = queryset.query.sql_with_params()
    
    # Format the SQL query
    formatted_sql = format_sql(sql % params)
    
    return formatted_sql

In [13]:
class FieldsFilter(BaseFilterBackend):
    def filter_queryset(self, request, queryset, view):
        """
        This performs field level filtering on the result set
        Eg: ?title=James Joyce
        """
        fields = set(view.get_available_fields(queryset.model, db_fields_only=True))

        # Locale is a database field, but we provide a separate filter for it
        if "locale" in fields:
            fields.remove("locale")

        for field_name, value in request.GET.items():
            if field_name in fields:
                try:
                    field = queryset.model._meta.get_field(field_name)
                except LookupError:
                    field = None

                # Convert value into python
                try:
                    if isinstance(
                        field, (models.BooleanField, models.NullBooleanField)
                    ):
                        value = parse_boolean(value)
                    elif isinstance(field, (models.IntegerField, models.AutoField)):
                        value = int(value)
                    elif isinstance(field, models.ForeignKey):
                        value = field.target_field.get_prep_value(value)
                except ValueError as e:
                    raise BadRequestError(
                        "field filter error. '%s' is not a valid value for %s (%s)"
                        % (value, field_name, str(e))
                    )

                if isinstance(field, TaggableManager):
                    for tag in value.split(","):
                        queryset = queryset.filter(**{field_name + "__name": tag})

                    # Stick a message on the queryset to indicate that tag filtering has been performed
                    # This will let the do_search method know that it must raise an error as searching
                    # and tag filtering at the same time is not supported
                    queryset._filtered_by_tag = True
                else:
                    print("value? ", value)
                    queryset = queryset.filter(**{field_name: value})

        return queryset

In [14]:
blog = plus.Blog.objects.first()
rf = APIRequestFactory()
pages_endpoint = reverse("cast:api:wagtail:pages:listing")
bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
url = f"{pages_endpoint}?slug={bad_input}"
request = rf.get(url)

In [15]:
for field_name, value in request.GET.items():
    print("name: ", field_name, "value: ", value)

name:  slug value:  1 ����%2527%2522


### Use ViewSet and FieldsFilter -> Crash

In [16]:
# queryset = Page.objects.all().live()
# viewset = FilteredPagesAPIViewSet()
# queryset = FieldsFilter().filter_queryset(request, queryset, viewset)
# print(get_formatted_sql(queryset))
# queryset.count()

### Use FieldsFilter and be ok

In [17]:
request.GET

<QueryDict: {'slug': ['1\x00����%2527%2522']}>

In [18]:
queryset = Page.objects.all().live()
# queryset = queryset.filter(**{"slug": "1����%2527%2522"})
queryset = queryset.filter(**{"slug": "1\x00����%2527%2522"})
# queryset = queryset.filter(**{"slug": request.GET["slug"]})
print(get_formatted_sql(queryset))
print(queryset.count())

SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title"

DataError: PostgreSQL text fields cannot contain NUL (0x00) bytes

## What About PostFilterset?

In [21]:
from django.test import RequestFactory
from cast.filters import PostFilterset

rf = RequestFactory()
bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
request = rf.get(f"?slug={bad_input}")
queryset = plus.Post.objects.all()
filterset = PostFilterset(request.GET, queryset=queryset)
print(filterset.qs.count())

58


## WTF...

In [7]:
rf = APIRequestFactory()
pages_endpoint = reverse("cast:api:wagtail:pages:listing")
bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
# bad_input = "devops-redux"
# url = f"{pages_endpoint}?child_of={blog.pk}&slug={bad_input}"
url = f"{pages_endpoint}?slug={bad_input}"
request = rf.get(url)
queryset = Page.objects.all().live()
viewset = FilteredPagesAPIViewSet()
# queryset = FieldsFilter().filter_queryset(request, queryset, viewset)
queryset = queryset.filter(**{"slug": "1����%2527%2522"})
print(get_formatted_sql(queryset))
print(queryset.count())
for page in queryset:
    print(page.pk, page.slug)

SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title"

In [7]:
query = """
SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title",
       "wagtailcore_page"."show_in_menus",
       "wagtailcore_page"."search_description",
       "wagtailcore_page"."latest_revision_created_at",
       "wagtailcore_page"."alias_of_id"
FROM "wagtailcore_page"
WHERE ("wagtailcore_page"."live"
       AND "wagtailcore_page"."slug" = 1����%2527%2522)
ORDER BY "wagtailcore_page"."path" ASC
"""
# params = ["devops-redux"]
params = ["1����%2527%2522"]

with connection.cursor() as cursor:
    cursor.execute(query, params)
    # cursor.execute(query)
    results = cursor.fetchall()

for row in results:
    print(row)

ProgrammingError: only '%s', '%b', '%t' are allowed as placeholders, got '%2'

In [ ]:
for page in plus.Post.objects.all():
    print(page.pk, page.slug)

In [7]:
for field_name, value in request.GET.items():
    print(field_name, value)

slug 1 ����%2527%2522
